In [1]:
import json
import pandas as pd
import numpy as np


In [58]:
import json
import pandas as pd

# Carrega o arquivo JSON
with open('data/raw/revendedores_combustivel_20250802_170416.json', encoding='utf-8') as f:
    data = json.load(f)

# Extrai a lista de registros
registros = data["data"]

#1. DataFrame de informações cadastrais
colunas_cadastrais = [
    'codigoSIMP', 'autorizacao', 'dataPublicacao', 'razaoSocial', 'cnpj',
    'endereco', 'complemento', 'bairro', 'cep', 'uf', 'municipio',
    'distribuidora', 'dataVinculacao', 'classe'
]

# Normaliza apenas os campos desejados
dados_cadastrais = pd.json_normalize(registros)[colunas_cadastrais]

#  2. DataFrame de produtos 
dados_produtos = pd.json_normalize(
    registros,
    record_path='produtos',
    meta=['codigoSIMP']
)

#  3. DataFrame de localização 
dados_geo = pd.json_normalize(registros)[[
    'codigoSIMP',
    'latitude', 'longitude', 'latitude_ANP4C', 'longitude_ANP4C',
    'validacao', 'estimativaAcuracia', 'srid', 'src',
    'dataObtencao', 'origemInformacao', 'situacaoConstatada',
    'observacao', 'statusSIGAF'
]]


In [59]:
dados_cadastrais.columns = dados_cadastrais.columns.str.lower()
dados_produtos.columns = dados_produtos.columns.str.lower()
dados_geo.columns = dados_geo.columns.str.lower()

In [60]:
# mudando para maiúsculas as informações de localização
dados_geo = dados_geo.applymap(lambda x: x.upper() if isinstance(x, str) else x)

C:\Users\cleto\AppData\Local\Temp\ipykernel_27540\2758304496.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dados_geo = dados_geo.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [61]:
# Convertendo a coluna 'tancagem' para inteiro
dados_produtos['tancagem'] = dados_produtos['tancagem'].astype(int)

In [62]:
# colunas que possui informações em código ou pouco relevantes
dados_geo = dados_geo.drop(columns=['origeminformacao', 'observacao', 'situacaoconstatada', 'srid'])

In [63]:
# da pra colocar NA aqui ou NaN 
dados_cadastrais.replace('', np.nan, inplace=True)
dados_produtos.replace('', np.nan, inplace=True)
dados_geo.replace('', np.nan, inplace=True)

In [ ]:
dados_cadastrais.to_json('data/processed/dados_cadastrais.json', orient='records', force_ascii=False, indent=4)
dados_produtos.to_json('data/processed/dados_produtos.json', orient='records', force_ascii=False, indent=4)
dados_geo.to_json('data/processed/dados_geo.json', orient='records', force_ascii=False, indent=4)


